In [3]:
import cv2 as cv
import glob
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [21]:
from keras.models import Model
from keras.layers import Input, concatenate, Convolution2D, MaxPooling2D, UpSampling2D,Lambda
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K

In [5]:
data_dir = '/Users/matt/masters_thesis/resized_data'

In [16]:
def load_files_names(data_dir):
    files = [f for f in glob.glob('{}/*.jpg'.format(data_dir)) if '_mask' not in f]
    files = list(
        map(
            lambda f_name: (f_name, '{0}_mask.jpg'.format(f_name.split('.')[0])),
            files
        )
    )
    return files

In [17]:
data_files = load_files_names(data_dir)

In [ ]:
class BatchGenerator(object):
    """Class generating image batches"""
    
    def __init__(self, data_dir):
        self.train_batch_pos = 0
        self.test_batch_pos = 0
        pass
    
    def load_data(self):
        pass
    
    @property
    def train_batch(self):
        pass
    
    @property
    def test_batch(self):
        pass

In [20]:
smooth = 1.
def IOU_calc(y_true, y_false):
    y_true_f = K.flatten(y_true)
    y_false_f = K.flatten(y_false)
    intersection = K.sum(y_true_f * y_false_f)
    return 2*(intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def IOU_loss(y_true, y_false):
    return -1 * IOU_calc(y_true, y_false)
    

In [22]:
def unet_prototype(img_rows, img_cols):
    inputs = Input(shape=(img_rows, img_cols, 3))
    
    conv1 = Convolution2D(filters=8, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    conv1 = Convolution2D(filters=8, kernel_size=(3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Convolution2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(pool1)
    conv2 = Convolution2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(pool2)
    conv3 = Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Convolution2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(pool3)
    conv4 = Convolution2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Convolution2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(pool4)
    conv5 = Convolution2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(conv5)
    
    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=-1)
    conv6 = Convolution2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(up6)
    conv6 = Convolution2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(conv6)
    
    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=-1)
    conv7 = Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(up7)
    conv7 = Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(conv7)
    
    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
    conv8 = Convolution2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(up8)
    conv8 = Convolution2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(conv8)
    
    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=-1)
    conv9 = Convolution2D(filters=8, kernel_size=(3, 3), activation='relu', padding='same')(up9)
    conv9 = Convolution2D(filters=8, kernel_size=(3, 3), activation='relu', padding='same')(conv9)
    
    conv10 = Convolution2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(conv9)
    
    model = Model(inputs=inputs, outputs=conv10)
    
    return model